# Insurance cost prediction using linear regression

Make a submisson here: https://jovian.ai/learn/deep-learning-with-pytorch-zero-to-gans/assignment/assignment-2-train-your-first-model

In this assignment we're going to use information like a person's age, sex, BMI, no. of children and smoking habit to predict the price of yearly medical bills. This kind of model is useful for insurance companies to determine the yearly insurance premium for a person. The dataset for this problem is taken from [Kaggle](https://www.kaggle.com/mirichoi0218/insurance).


We will create a model with the following steps:
1. Download and explore the dataset
2. Prepare the dataset for training
3. Create a linear regression model
4. Train the model to fit the data
5. Make predictions using the trained model


This assignment builds upon the concepts from the first 2 lessons. It will help to review these Jupyter notebooks:
- PyTorch basics: https://jovian.ai/aakashns/01-pytorch-basics
- Linear Regression: https://jovian.ai/aakashns/02-linear-regression
- Logistic Regression: https://jovian.ai/aakashns/03-logistic-regression
- Linear regression (minimal): https://jovian.ai/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ai/aakashns/mnist-logistic-minimal

As you go through this notebook, you will find a **???** in certain places. Your job is to replace the **???** with appropriate code or values, to ensure that the notebook runs properly end-to-end . In some cases, you'll be required to choose some hyperparameters (learning rate, batch size etc.). Try to experiment with the hypeparameters to get the lowest loss.


In [4]:
# Uncomment and run the appropriate command for your operating system, if required

# Linux / Binder
# !pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Windows
# !pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# MacOS
# !pip install numpy matplotlib pandas torch torchvision torchaudio

In [5]:
import torch
import jovian
import torchvision
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split

In [6]:
project_name='02-insurance-linear-regression' # will be used by jovian.commit

## Step 1: Download and explore the data

Let us begin by downloading the data. We'll use the `download_url` function from PyTorch to get the data as a CSV (comma-separated values) file. 

In [7]:
DATASET_URL = "https://hub.jovian.ml/wp-content/uploads/2020/05/insurance.csv"
DATA_FILENAME = "insurance.csv"
download_url(DATASET_URL, '.')

To load the dataset into memory, we'll use the `read_csv` function from the `pandas` library. The data will be loaded as a Pandas dataframe. See this short tutorial to learn more: https://data36.com/pandas-tutorial-1-basics-reading-data-files-dataframes-data-selection/

In [8]:
dataframe_raw = pd.read_csv(DATA_FILENAME)
dataframe_raw.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


We're going to do a slight customization of the data, so that you every participant receives a slightly different version of the dataset. Fill in your name below as a string (enter at least 5 characters)

In [9]:
your_name = 'YasHv' # at least 5 characters

The `customize_dataset` function will customize the dataset slightly using your name as a source of random numbers.

In [10]:
def customize_dataset(dataframe_raw, rand_str):
    dataframe = dataframe_raw.copy(deep=True)
    # drop some rows
    dataframe = dataframe.sample(int(0.95*len(dataframe)), random_state=int(ord(rand_str[0])))
    # scale input
    dataframe.bmi = dataframe.bmi * ord(rand_str[1])/100.
    # scale target
    dataframe.charges = dataframe.charges * ord(rand_str[2])/100.
    # drop column
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop([' region '], axis=1)
    return dataframe

In [11]:
dataframe=dataframe_raw.copy()
dataframe=dataframe.drop(["region"], axis=1)
dataframe.head()

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520


In [12]:
dataframe = customize_dataset(dataframe_raw, your_name)
dataframe.head()

,age,sex,bmi,children,smoker,region,charges
290,28,female,32.39800,0,no,southwest,3647.820700
651,53,female,38.41200,1,no,southeast,12166.667650
118,49,female,26.35490,0,no,southeast,9891.528695
255,55,female,24.60405,3,no,northeast,15004.432203
1166,57,male,39.15890,0,no,southeast,12629.876495


Let us answer some basic questions about the dataset. 


**Q: How many rows does the dataset have?**

In [13]:
num_rows = len(dataframe)
print(num_rows)

1271


**Q: How many columns doe the dataset have**

In [14]:
num_cols = len(dataframe.columns)
print(num_cols)

7


**Q: What are the column titles of the input variables?**

In [15]:
input_cols = list(dataframe.columns)[:-1]
input_cols

['age', 'sex', 'bmi', 'children', 'smoker', 'region']

**Q: Which of the input columns are non-numeric or categorial variables ?**

Hint: `sex` is one of them. List the columns that are not numbers.

In [16]:
categorical_cols = dataframe.select_dtypes(exclude=['number']).columns
categorical_cols

Index(['sex', 'smoker', 'region'], dtype='object')

**Q: What are the column titles of output/target variable(s)?**

In [17]:
output_cols = [dataframe.columns[-1]]
output_cols

['charges']

**Q: (Optional) What is the minimum, maximum and average value of the `charges` column? Can you show the distribution of values in a graph?**
Use this data visualization cheatsheet for referece: https://jovian.ml/aakashns/dataviz-cheatsheet

In [18]:
# Write your answer here
print("Minimum value:- {}".format(dataframe['charges'].min()))
print("Maximum value:-  {}".format(dataframe['charges'].max()))
print("Mean value:- {}".format(dataframe['charges'].mean()))

Minimum value:- 1290.1549850000001
Maximum value:-  73335.99221150001
Mean value:- 15386.991450052219


Remember to commit your notebook to Jovian after every step, so that you don't lose your work.

In [19]:
!pip install jovian --upgrade -q

In [20]:
import jovian

In [21]:
jovian.commit(project=project_name)

[jovian] Detected Colab notebook...
[jovian] Please enter your API key ( from https://jovian.ai/ ):
API KEY: ··········
[jovian] Uploading colab notebook to Jovian...
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ai/vardhanyash20000/02-insurance-linear-regression


'https://jovian.ai/vardhanyash20000/02-insurance-linear-regression'

## Step 2: Prepare the dataset for training

We need to convert the data from the Pandas dataframe into a PyTorch tensors for training. To do this, the first step is to convert it numpy arrays. If you've filled out `input_cols`, `categorial_cols` and `output_cols` correctly, this following function will perform the conversion to numpy arrays.

In [22]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

Read through the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) to understand how we're converting categorical variables into numbers.

In [23]:
inputs_array, targets_array = dataframe_to_arrays(dataframe)
inputs_array, targets_array

(array([[28.    ,  0.    , 32.398 ,  0.    ,  0.    ,  3.    ],
        [53.    ,  0.    , 38.412 ,  1.    ,  0.    ,  2.    ],
        [49.    ,  0.    , 26.3549,  0.    ,  0.    ,  2.    ],
        ...,
        [53.    ,  1.    , 40.2259,  0.    ,  0.    ,  2.    ],
        [40.    ,  0.    , 27.8293,  3.    ,  0.    ,  1.    ],
        [60.    ,  1.    , 23.5904,  0.    ,  0.    ,  1.    ]]),
 array([[ 3647.8207  ],
        [12166.66765 ],
        [ 9891.528695],
        ...,
        [10929.956845],
        [ 9268.630965],
        [14402.14552 ]]))

**Q: Convert the numpy arrays `inputs_array` and `targets_array` into PyTorch tensors. Make sure that the data type is `torch.float32`.**

In [24]:
inputs = torch.from_numpy(inputs_array).float()
targets = torch.from_numpy(targets_array).float()

In [25]:
inputs.dtype, targets.dtype

(torch.float32, torch.float32)

Next, we need to create PyTorch datasets & data loaders for training & validation. We'll start by creating a `TensorDataset`.

In [26]:
dataset = TensorDataset(inputs, targets)

**Q: Pick a number between `0.1` and `0.2` to determine the fraction of data that will be used for creating the validation set. Then use `random_split` to create training & validation datasets.**

In [27]:
val_percent = 0.12 # between 0.1 and 0.2
val_size = int(num_rows * val_percent)
train_size = num_rows - val_size


train_ds, val_ds = random_split(dataset, [train_size, val_size]) # Use the random_split function to split dataset into 2 parts of the desired length

Finally, we can create data loaders for training & validation.

**Q: Pick a batch size for the data loader.**

In [28]:
batch_size = 60

In [29]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

Let's look at a batch of data to verify everything is working fine so far.

In [30]:
for xb, yb in train_loader:
    print("inputs:", xb)
    print("targets:", yb)
    break

inputs: tensor([[32.0000,  0.0000, 28.8430,  0.0000,  0.0000,  1.0000],
        [60.0000,  0.0000, 23.3139,  0.0000,  0.0000,  1.0000],
        [48.0000,  0.0000, 22.1160,  0.0000,  0.0000,  3.0000],
        [19.0000,  1.0000, 20.2730,  1.0000,  0.0000,  3.0000],
        [38.0000,  0.0000, 27.1600,  3.0000,  0.0000,  3.0000],
        [28.0000,  1.0000, 35.3080,  1.0000,  1.0000,  3.0000],
        [33.0000,  1.0000, 34.1876,  0.0000,  0.0000,  0.0000],
        [56.0000,  0.0000, 34.7260,  1.0000,  0.0000,  3.0000],
        [43.0000,  1.0000, 25.2491,  0.0000,  0.0000,  0.0000],
        [55.0000,  0.0000, 34.1440,  0.0000,  1.0000,  2.0000],
        [28.0000,  1.0000, 23.5710,  5.0000,  0.0000,  3.0000],
        [28.0000,  0.0000, 23.5904,  1.0000,  0.0000,  0.0000],
        [37.0000,  0.0000, 46.1720,  2.0000,  1.0000,  3.0000],
        [57.0000,  1.0000, 39.7166,  0.0000,  0.0000,  0.0000],
        [30.0000,  1.0000, 30.4580,  1.0000,  0.0000,  3.0000],
        [34.0000,  0.0000, 28.38

Let's save our work by committing to Jovian.

In [31]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Committed successfully! https://jovian.ai/vardhanyash20000/02-insurance-linear-regression


'https://jovian.ai/vardhanyash20000/02-insurance-linear-regression'

## Step 3: Create a Linear Regression Model

Our model itself is a fairly straightforward linear regression (we'll build more complex models in the next assignment). 


In [32]:
input_size = len(input_cols)
output_size = len(output_cols)

**Q: Complete the class definition below by filling out the constructor (`__init__`), `forward`, `training_step` and `validation_step` methods.**

Hint: Think carefully about picking a good loss fuction (it's not cross entropy). Maybe try 2-3 of them and see which one works best. See https://pytorch.org/docs/stable/nn.functional.html#loss-functions

In [33]:
class InsuranceModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)                  # fill this (hint: use input_size & output_size defined above)
        
    def forward(self, xb):
        out = self.linear(xb)                         # fill this
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        loss = F.l1_loss(out, targets)                          # fill this
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.l1_loss(out, targets)                          # fill this    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

Let us create a model using the `InsuranceModel` class. You may need to come back later and re-run the next cell to reinitialize the model, in case the loss becomes `nan` or `infinity`.

In [34]:
model = InsuranceModel()

Let's check out the weights and biases of the model using `model.parameters`.

In [35]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.4028, -0.1777,  0.2651, -0.2925,  0.0241,  0.2872]],
        requires_grad=True), Parameter containing:
 tensor([-0.0637], requires_grad=True)]

One final commit before we train the model.

In [36]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Committed successfully! https://jovian.ai/vardhanyash20000/02-insurance-linear-regression


'https://jovian.ai/vardhanyash20000/02-insurance-linear-regression'

## Step 4: Train the model to fit the data

To train our model, we'll use the same `fit` function explained in the lecture. That's the benefit of defining a generic training loop - you can use it for any problem.

In [37]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

**Q: Use the `evaluate` function to calculate the loss on the validation set before training.**

In [58]:
result =  evaluate(model,val_loader) # Use the the evaluate function
print(result)

{'val_loss': 3348.004150390625}



We are now ready to train the model. You may need to run the training loop many times, for different number of epochs and with different learning rates, to get a good result. Also, if your loss becomes too large (or `nan`), you may have to re-initialize the model by running the cell `model = InsuranceModel()`. Experiment with this for a while, and try to get to as low a loss as possible.

**Q: Train the model 4-5 times with different learning rates & for different number of epochs.**

Hint: Vary learning rates by orders of 10 (e.g. `1e-2`, `1e-3`, `1e-4`, `1e-5`, `1e-6`) to figure out what works.

In [39]:
epochs = 1100
lr = 2e-2
history2 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 7320.7134
Epoch [40], val_loss: 7175.2036
Epoch [60], val_loss: 7069.3042
Epoch [80], val_loss: 6972.8809
Epoch [100], val_loss: 6883.7563
Epoch [120], val_loss: 6797.2480
Epoch [140], val_loss: 6718.6860
Epoch [160], val_loss: 6653.9727
Epoch [180], val_loss: 6598.9590
Epoch [200], val_loss: 6560.1777
Epoch [220], val_loss: 6532.3091
Epoch [240], val_loss: 6512.7344
Epoch [260], val_loss: 6503.8745
Epoch [280], val_loss: 6493.1030
Epoch [300], val_loss: 6491.5742
Epoch [320], val_loss: 6490.7983
Epoch [340], val_loss: 6492.4438
Epoch [360], val_loss: 6490.8530
Epoch [380], val_loss: 6488.6772
Epoch [400], val_loss: 6486.9609
Epoch [420], val_loss: 6486.0015
Epoch [440], val_loss: 6484.6890
Epoch [460], val_loss: 6484.1406
Epoch [480], val_loss: 6484.4019
Epoch [500], val_loss: 6481.4800
Epoch [520], val_loss: 6479.2891
Epoch [540], val_loss: 6478.7734
Epoch [560], val_loss: 6477.4077
Epoch [580], val_loss: 6476.8809
Epoch [600], val_loss: 6475.6138
Epoch [620], v

In [40]:
epochs = 1100
lr = 5e-1
history3 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 6400.5532
Epoch [40], val_loss: 6390.0000
Epoch [60], val_loss: 6388.0884
Epoch [80], val_loss: 6352.5469
Epoch [100], val_loss: 6381.7773
Epoch [120], val_loss: 6317.3008
Epoch [140], val_loss: 6307.5913
Epoch [160], val_loss: 6333.4648
Epoch [180], val_loss: 6290.2593
Epoch [200], val_loss: 6278.1333
Epoch [220], val_loss: 6269.7524
Epoch [240], val_loss: 6296.8726
Epoch [260], val_loss: 6268.2266
Epoch [280], val_loss: 6242.1426
Epoch [300], val_loss: 6313.6235
Epoch [320], val_loss: 6224.7441
Epoch [340], val_loss: 6268.0405
Epoch [360], val_loss: 6210.3574
Epoch [380], val_loss: 6233.0815
Epoch [400], val_loss: 6193.8491
Epoch [420], val_loss: 6180.8062
Epoch [440], val_loss: 6172.2520
Epoch [460], val_loss: 6189.2202
Epoch [480], val_loss: 6156.7148
Epoch [500], val_loss: 6149.6489
Epoch [520], val_loss: 6142.2363
Epoch [540], val_loss: 6210.0547
Epoch [560], val_loss: 6223.5220
Epoch [580], val_loss: 6119.4917
Epoch [600], val_loss: 6126.5093
Epoch [620], v

In [41]:
epochs = 1100
lr = 3e-1
history4 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 5905.1655
Epoch [40], val_loss: 5880.3325
Epoch [60], val_loss: 5878.9038
Epoch [80], val_loss: 5870.1577
Epoch [100], val_loss: 5870.7285
Epoch [120], val_loss: 5847.0874
Epoch [140], val_loss: 5841.1953
Epoch [160], val_loss: 5853.4644
Epoch [180], val_loss: 5894.4258
Epoch [200], val_loss: 5863.7407
Epoch [220], val_loss: 5829.4946
Epoch [240], val_loss: 5878.8052
Epoch [260], val_loss: 5820.7358
Epoch [280], val_loss: 5803.9688
Epoch [300], val_loss: 5810.0396
Epoch [320], val_loss: 5818.3892
Epoch [340], val_loss: 5793.3901
Epoch [360], val_loss: 5839.8120
Epoch [380], val_loss: 5833.5410
Epoch [400], val_loss: 5780.9048
Epoch [420], val_loss: 5777.5059
Epoch [440], val_loss: 5774.6548
Epoch [460], val_loss: 5767.5176
Epoch [480], val_loss: 5774.2969
Epoch [500], val_loss: 5759.2168
Epoch [520], val_loss: 5742.7446
Epoch [540], val_loss: 5760.2949
Epoch [560], val_loss: 5739.5913
Epoch [580], val_loss: 5765.0039
Epoch [600], val_loss: 5775.6094
Epoch [620], v

In [42]:
epochs = 1100
lr = 5e-1
history5 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 5585.2710
Epoch [40], val_loss: 5599.7974
Epoch [60], val_loss: 5566.3589
Epoch [80], val_loss: 5602.2847
Epoch [100], val_loss: 5596.8359
Epoch [120], val_loss: 5548.2720
Epoch [140], val_loss: 5537.4570
Epoch [160], val_loss: 5532.2793
Epoch [180], val_loss: 5536.0962
Epoch [200], val_loss: 5516.3989
Epoch [220], val_loss: 5519.2856
Epoch [240], val_loss: 5541.4360
Epoch [260], val_loss: 5481.8374
Epoch [280], val_loss: 5571.3516
Epoch [300], val_loss: 5465.3120
Epoch [320], val_loss: 5456.8765
Epoch [340], val_loss: 5488.4116
Epoch [360], val_loss: 5469.6113
Epoch [380], val_loss: 5520.2773
Epoch [400], val_loss: 5424.8032
Epoch [420], val_loss: 5416.2954
Epoch [440], val_loss: 5409.7168
Epoch [460], val_loss: 5480.6069
Epoch [480], val_loss: 5487.4214
Epoch [500], val_loss: 5394.2388
Epoch [520], val_loss: 5399.0435
Epoch [540], val_loss: 5366.0093
Epoch [560], val_loss: 5370.1479
Epoch [580], val_loss: 5348.5874
Epoch [600], val_loss: 5349.4321
Epoch [620], v

In [43]:
epochs = 1100
lr = 3e-2
history5 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 5138.2593
Epoch [40], val_loss: 5138.3433
Epoch [60], val_loss: 5136.2183
Epoch [80], val_loss: 5134.5293
Epoch [100], val_loss: 5137.2700
Epoch [120], val_loss: 5141.8252
Epoch [140], val_loss: 5136.9980
Epoch [160], val_loss: 5137.6382
Epoch [180], val_loss: 5135.6519
Epoch [200], val_loss: 5135.0200
Epoch [220], val_loss: 5133.4341
Epoch [240], val_loss: 5137.8145
Epoch [260], val_loss: 5133.5317
Epoch [280], val_loss: 5134.3984
Epoch [300], val_loss: 5133.0327
Epoch [320], val_loss: 5133.8726
Epoch [340], val_loss: 5134.6030
Epoch [360], val_loss: 5131.4907
Epoch [380], val_loss: 5132.5156
Epoch [400], val_loss: 5131.4536
Epoch [420], val_loss: 5131.2480
Epoch [440], val_loss: 5128.3276
Epoch [460], val_loss: 5129.4043
Epoch [480], val_loss: 5126.0786
Epoch [500], val_loss: 5128.4590
Epoch [520], val_loss: 5126.8491
Epoch [540], val_loss: 5124.0449
Epoch [560], val_loss: 5125.3062
Epoch [580], val_loss: 5125.5923
Epoch [600], val_loss: 5125.0381
Epoch [620], v

**Q: What is the final validation loss of your model?**

In [44]:
epochs = 1100
lr = 1e-1
history5 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 5102.6836
Epoch [40], val_loss: 5108.1558
Epoch [60], val_loss: 5099.0356
Epoch [80], val_loss: 5112.3530
Epoch [100], val_loss: 5102.3589
Epoch [120], val_loss: 5107.2666
Epoch [140], val_loss: 5099.0234
Epoch [160], val_loss: 5104.9824
Epoch [180], val_loss: 5093.0679
Epoch [200], val_loss: 5091.6230
Epoch [220], val_loss: 5108.8472
Epoch [240], val_loss: 5103.6201
Epoch [260], val_loss: 5099.5239
Epoch [280], val_loss: 5084.8120
Epoch [300], val_loss: 5088.3799
Epoch [320], val_loss: 5083.8579
Epoch [340], val_loss: 5081.8486
Epoch [360], val_loss: 5078.7539
Epoch [380], val_loss: 5076.8862
Epoch [400], val_loss: 5076.2744
Epoch [420], val_loss: 5074.3540
Epoch [440], val_loss: 5069.6440
Epoch [460], val_loss: 5069.6509
Epoch [480], val_loss: 5076.4458
Epoch [500], val_loss: 5070.4785
Epoch [520], val_loss: 5070.3765
Epoch [540], val_loss: 5072.9604
Epoch [560], val_loss: 5068.7949
Epoch [580], val_loss: 5063.4980
Epoch [600], val_loss: 5059.6060
Epoch [620], v

In [45]:
epochs = 1100
lr = 5e-1
history5 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 5006.3145
Epoch [40], val_loss: 4998.9810
Epoch [60], val_loss: 4997.2173
Epoch [80], val_loss: 4992.0771
Epoch [100], val_loss: 4972.7856
Epoch [120], val_loss: 4974.1821
Epoch [140], val_loss: 4956.2300
Epoch [160], val_loss: 4954.8540
Epoch [180], val_loss: 4972.0229
Epoch [200], val_loss: 4936.1743
Epoch [220], val_loss: 4922.8521
Epoch [240], val_loss: 5032.5737
Epoch [260], val_loss: 4916.5298
Epoch [280], val_loss: 4963.0801
Epoch [300], val_loss: 4935.9224
Epoch [320], val_loss: 4883.2563
Epoch [340], val_loss: 4941.9985
Epoch [360], val_loss: 4905.1138
Epoch [380], val_loss: 4934.8232
Epoch [400], val_loss: 4944.1484
Epoch [420], val_loss: 4859.7549
Epoch [440], val_loss: 4833.1851
Epoch [460], val_loss: 4824.1978
Epoch [480], val_loss: 4817.6421
Epoch [500], val_loss: 4830.1421
Epoch [520], val_loss: 4836.8325
Epoch [540], val_loss: 4825.1401
Epoch [560], val_loss: 4787.6606
Epoch [580], val_loss: 4777.0762
Epoch [600], val_loss: 4769.6758
Epoch [620], v

In [46]:
epochs = 1100
lr = 3e-1
history5 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 4611.1641
Epoch [40], val_loss: 4567.2827
Epoch [60], val_loss: 4570.7456
Epoch [80], val_loss: 4547.5581
Epoch [100], val_loss: 4540.0103
Epoch [120], val_loss: 4549.4829
Epoch [140], val_loss: 4531.4756
Epoch [160], val_loss: 4524.5586
Epoch [180], val_loss: 4538.7554
Epoch [200], val_loss: 4523.1177
Epoch [220], val_loss: 4510.1987
Epoch [240], val_loss: 4518.8862
Epoch [260], val_loss: 4502.6470
Epoch [280], val_loss: 4506.1938
Epoch [300], val_loss: 4492.5820
Epoch [320], val_loss: 4485.9160
Epoch [340], val_loss: 4505.0596
Epoch [360], val_loss: 4493.1812
Epoch [380], val_loss: 4494.0645
Epoch [400], val_loss: 4479.9321
Epoch [420], val_loss: 4469.8716
Epoch [440], val_loss: 4458.3906
Epoch [460], val_loss: 4462.3809
Epoch [480], val_loss: 4449.3892
Epoch [500], val_loss: 4471.2153
Epoch [520], val_loss: 4436.8257
Epoch [540], val_loss: 4437.3013
Epoch [560], val_loss: 4437.8970
Epoch [580], val_loss: 4422.2729
Epoch [600], val_loss: 4442.1968
Epoch [620], v

In [47]:
epochs = 1100
lr = 5e-1
history5 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 4288.8442
Epoch [40], val_loss: 4322.7554
Epoch [60], val_loss: 4363.3091
Epoch [80], val_loss: 4265.4741
Epoch [100], val_loss: 4338.9399
Epoch [120], val_loss: 4248.8267
Epoch [140], val_loss: 4244.0576
Epoch [160], val_loss: 4239.1792
Epoch [180], val_loss: 4249.0454
Epoch [200], val_loss: 4226.2227
Epoch [220], val_loss: 4218.2432
Epoch [240], val_loss: 4205.0854
Epoch [260], val_loss: 4251.0396
Epoch [280], val_loss: 4186.0044
Epoch [300], val_loss: 4193.1030
Epoch [320], val_loss: 4184.5815
Epoch [340], val_loss: 4164.1880
Epoch [360], val_loss: 4169.3433
Epoch [380], val_loss: 4172.7231
Epoch [400], val_loss: 4145.2173
Epoch [420], val_loss: 4137.9995
Epoch [440], val_loss: 4137.7407
Epoch [460], val_loss: 4119.1128
Epoch [480], val_loss: 4113.8408
Epoch [500], val_loss: 4100.9233
Epoch [520], val_loss: 4088.0645
Epoch [540], val_loss: 4120.5005
Epoch [560], val_loss: 4084.7197
Epoch [580], val_loss: 4084.8545
Epoch [600], val_loss: 4103.3921
Epoch [620], v

In [48]:
epochs = 1100
lr = 3e-1
history5 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 3883.7991
Epoch [40], val_loss: 3865.6992
Epoch [60], val_loss: 3855.6863
Epoch [80], val_loss: 3854.9651
Epoch [100], val_loss: 3852.3875
Epoch [120], val_loss: 3887.0994
Epoch [140], val_loss: 3846.2227
Epoch [160], val_loss: 3840.8625
Epoch [180], val_loss: 3842.7507
Epoch [200], val_loss: 3842.8699
Epoch [220], val_loss: 3820.7947
Epoch [240], val_loss: 3827.2239
Epoch [260], val_loss: 3829.3142
Epoch [280], val_loss: 3834.5820
Epoch [300], val_loss: 3818.3005
Epoch [320], val_loss: 3804.1311
Epoch [340], val_loss: 3810.5706
Epoch [360], val_loss: 3931.8562
Epoch [380], val_loss: 3788.1416
Epoch [400], val_loss: 3800.4182
Epoch [420], val_loss: 3779.5381
Epoch [440], val_loss: 3818.9915
Epoch [460], val_loss: 3771.8750
Epoch [480], val_loss: 3771.3054
Epoch [500], val_loss: 3782.5898
Epoch [520], val_loss: 3761.0149
Epoch [540], val_loss: 3759.8418
Epoch [560], val_loss: 3758.8030
Epoch [580], val_loss: 3779.6726
Epoch [600], val_loss: 3741.5203
Epoch [620], v

In [49]:
epochs = 1100
lr = 5e-1
history5 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 3761.0439
Epoch [40], val_loss: 3628.4153
Epoch [60], val_loss: 3622.5574
Epoch [80], val_loss: 3640.0535
Epoch [100], val_loss: 3610.2590
Epoch [120], val_loss: 3625.1289
Epoch [140], val_loss: 3614.8459
Epoch [160], val_loss: 3665.5544
Epoch [180], val_loss: 3593.7781
Epoch [200], val_loss: 3580.3025
Epoch [220], val_loss: 3570.1897
Epoch [240], val_loss: 3597.2939
Epoch [260], val_loss: 3557.8425
Epoch [280], val_loss: 3552.4463
Epoch [300], val_loss: 3546.9148
Epoch [320], val_loss: 3539.6428
Epoch [340], val_loss: 3533.5938
Epoch [360], val_loss: 3541.5310
Epoch [380], val_loss: 3521.2639
Epoch [400], val_loss: 3576.2100
Epoch [420], val_loss: 3509.2375
Epoch [440], val_loss: 3504.8279
Epoch [460], val_loss: 3515.9133
Epoch [480], val_loss: 3505.6824
Epoch [500], val_loss: 3486.5830
Epoch [520], val_loss: 3504.6875
Epoch [540], val_loss: 3507.6638
Epoch [560], val_loss: 3469.7949
Epoch [580], val_loss: 3465.3430
Epoch [600], val_loss: 3618.9773
Epoch [620], v

In [59]:
val_loss = 3348

Let's log the final validation loss to Jovian and commit the notebook

In [60]:
jovian.log_metrics(val_loss=val_loss)

[jovian] Metrics logged.


In [61]:
jovian.commit(project=project_name, environment=None)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/vardhanyash20000/02-insurance-linear-regression


'https://jovian.ai/vardhanyash20000/02-insurance-linear-regression'

Now scroll back up, re-initialize the model, and try different set of values for batch size, number of epochs, learning rate etc. Commit each experiment and use the "Compare" and "View Diff" options on Jovian to compare the different results.

## Step 5: Make predictions using the trained model

**Q: Complete the following function definition to make predictions on a single input**

In [53]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)                # fill this
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

In [54]:
input, target = val_ds[0]
predict_single(input, target, model)

Input: tensor([51.0000,  0.0000, 17.5085,  0.0000,  0.0000,  1.0000])
Target: tensor([11090.8906])
Prediction: tensor([11987.7754])


In [55]:
input, target = val_ds[10]
predict_single(input, target, model)

Input: tensor([58.0000,  1.0000, 24.4198,  0.0000,  0.0000,  0.0000])
Target: tensor([13720.7939])
Prediction: tensor([13793.5801])


In [56]:
input, target = val_ds[23]
predict_single(input, target, model)

Input: tensor([30.0000,  0.0000, 41.8264,  2.0000,  0.0000,  2.0000])
Target: tensor([5466.6821])
Prediction: tensor([6117.2256])


In [62]:
input, target = val_ds[66]
predict_single(input, target, model)

Input: tensor([34.0000,  1.0000, 24.5119,  1.0000,  0.0000,  1.0000])
Target: tensor([5628.9663])
Prediction: tensor([6787.5039])


In [63]:
input, target = val_ds[50]
predict_single(input, target, model)

Input: tensor([39.0000,  1.0000, 34.2410,  2.0000,  1.0000,  3.0000])
Target: tensor([46119.4727])
Prediction: tensor([22926.4434])


In [64]:
input, target = val_ds[90]
predict_single(input, target, model)

Input: tensor([51.0000,  1.0000, 22.5137,  1.0000,  1.0000,  2.0000])
Target: tensor([25550.8320])
Prediction: tensor([26437.7344])


In [65]:
input, target = val_ds[44]
predict_single(input, target, model)

Input: tensor([48.0000,  0.0000, 32.1167,  0.0000,  1.0000,  2.0000])
Target: tensor([47120.2891])
Prediction: tensor([25503.1895])


Are you happy with your model's predictions? Try to improve them further.

## (Optional) Step 6: Try another dataset & blog about it

While this last step is optional for the submission of your assignment, we highly recommend that you do it. Try to replicate this notebook for a different linear regression or logistic regression problem. This will help solidify your understanding, and give you a chance to differentiate the generic patterns in machine learning from problem-specific details.You can use one of these starer notebooks (just change the dataset):

- Linear regression (minimal): https://jovian.ai/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ai/aakashns/mnist-logistic-minimal

Here are some sources to find good datasets:

- https://lionbridge.ai/datasets/10-open-datasets-for-linear-regression/
- https://www.kaggle.com/rtatman/datasets-for-regression-analysis
- https://archive.ics.uci.edu/ml/datasets.php?format=&task=reg&att=&area=&numAtt=&numIns=&type=&sort=nameUp&view=table
- https://people.sc.fsu.edu/~jburkardt/datasets/regression/regression.html
- https://archive.ics.uci.edu/ml/datasets/wine+quality
- https://pytorch.org/docs/stable/torchvision/datasets.html

We also recommend that you write a blog about your approach to the problem. Here is a suggested structure for your post (feel free to experiment with it):

- Interesting title & subtitle
- Overview of what the blog covers (which dataset, linear regression or logistic regression, intro to PyTorch)
- Downloading & exploring the data
- Preparing the data for training
- Creating a model using PyTorch
- Training the model to fit the data
- Your thoughts on how to experiment with different hyperparmeters to reduce loss
- Making predictions using the model

As with the previous assignment, you can [embed Juptyer notebook cells & outputs from Jovian](https://medium.com/jovianml/share-and-embed-jupyter-notebooks-online-with-jovian-ml-df709a03064e) into your blog. 

Don't forget to share your work on the forum: https://jovian.ai/forum/t/linear-regression-and-logistic-regression-notebooks-and-blog-posts/14039

In [66]:
jovian.commit(project=project_name, environment=None)
jovian.commit(project=project_name, environment=None) # try again, kaggle fails sometimes

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/vardhanyash20000/02-insurance-linear-regression
[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/vardhanyash20000/02-insurance-linear-regression


'https://jovian.ai/vardhanyash20000/02-insurance-linear-regression'

In [67]:
jovian.submit(assignment="zerotogans-a2")

[jovian] Detected Colab notebook...


[jovian] Error: Please provide the project argument e.g. jovian.commit(project='my-project')
